### We will be using Twitter's official Python package to interact with the API. We encourage you to explore their source files, even if they seem a bit intimidating.  
https://github.com/twitterdev/search-tweets-python

In [1]:
#This function installs Twitter's Python API software
!pip install searchtweets-v2

#This function installs a progress visualizer
!pip install tqdm

You should consider upgrading via the 'c:\users\wilmd\onedrive\desktop\python_env\env\scripts\python.exe -m pip install --upgrade pip' command.



You should consider upgrading via the 'c:\users\wilmd\onedrive\desktop\python_env\env\scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import searchtweets

#This function should tell you that you are using version 1.1.1
print(searchtweets.__version__)

from tqdm import tqdm
from searchtweets import load_credentials
from searchtweets import gen_request_parameters
import pandas as pd

1.1.1


#### Edit the "creds.yaml" file in Jupyter by double clicking on it in the left pannel. Copy your information from your Twitter dev account:  
https://developer.twitter.com/en/portal/dashboard

In [78]:
#This line imports your credentials including your keys and bearer token
premium_search_args = load_credentials(filename="./creds.yaml", yaml_key="search_tweets_v2", env_overwrite = False)

#This line creates an empty list in which we will be storing our search results
tweet_storage = []

### Here we are specifying our search terms and time window!

In [75]:
start_date = "2018-01-01" #first date of desired search window 
end_date = "2023-02-06" #last date of desired search window
max_tweet_count = 10 #The number of Tweets you want to receive for a query

# For Geo-specific Tweets  
For geo-specific data, use a bounding box method which allows you to collect all geo-tagged Tweets within a certain geographic box.  The following website is a great tool to easily draw a bounding box.  
https://boundingbox.klokantech.com  
Use the 'csv' function in the drop down to copy and paste the coordinates.
add the following string to your query:
bounding_box:[-86.615353 39.121804 -86.464291 39.190223]

In [ ]:
query = ""

rule = gen_request_parameters(query, 
                              granularity = False, 
                              start_time = start_date, 
                              end_time = end_date,
                              expansions = "author_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id",
                              #Below are the columns which will be returned for your data set. Find more information on these columns here: https://github.com/twitterdev/search-tweets-python/blob/master/examples/api_example.ipynb
                              #This list covers the vast majority of all possible information you can gather from a Tweet.
                              tweet_fields='attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source'
                             )

In [83]:
next_token = None
#This line actually executes to search and adds it to the "tweet_storage" list.
while True:
    if next_token:
        response = searchtweets.collect_results(next_token, 
                                                max_tweets = max_tweet_count, 
                                                result_stream_args=premium_search_args)
        tweet_storage.append(response)
    else:
        response = searchtweets.collect_results(rule, 
                                                max_tweets = max_tweet_count, 
                                                result_stream_args=premium_search_args)
        tweet_storage.append(response)
    try:
        next_token = response['meta']['next_token']
    except:
        break


In [84]:
#Breaking down our data structure into a spreadsheet-like dataframe
tweet_flat = []
for i in tweet_storage:
    for j in i:
        if j.get('data'):
            for k in j['data']:
                tweet_flat.append(k)

In [85]:
#view a few of our tweets
pd.DataFrame(tweet_flat)

,lang,edit_history_tweet_ids,author_id,entities,conversation_id,reply_settings,text,id,public_metrics,created_at,possibly_sensitive,context_annotations,referenced_tweets,in_reply_to_user_id,attachments
0,en,[1548039919087415298],123883921,"{'hashtags': [{'start': 199, 'end': 202, 'tag'...",1548039919087415298,everyone,Just bought 2 rotors for my bike (posh ice tec...,1548039919087415298,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-07-15T20:20:52.000Z,False,NaN,NaN,NaN,NaN
1,en,[1185289088607440896],1147108534850658304,"{'annotations': [{'start': 100, 'end': 102, 'p...",1185289088607440896,everyone,RT @KeithTse2: Let's sign the #petition to @Gi...,1185289088607440896,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",2019-10-18T20:18:34.000Z,False,"[{'domain': {'id': '10', 'name': 'Person', 'de...","[{'type': 'retweeted', 'id': '1185288587442556...",NaN,NaN
2,en,[1185288587442556928],861942444,"{'annotations': [{'start': 85, 'end': 87, 'pro...",1185288587442556928,everyone,Let's sign the #petition to @GitHub @natfriedm...,1185288587442556928,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",2019-10-18T20:16:35.000Z,False,"[{'domain': {'id': '10', 'name': 'Person', 'de...",NaN,NaN,NaN
3,en,[1608909092621545473],97133219,"{'annotations': [{'start': 4, 'end': 7, 'proba...",1608909091400994817,everyone,"(2) OCAD members, filed a lawsuit with @ConMij...",1608909092621545473,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-12-30T19:33:14.000Z,False,"[{'domain': {'id': '30', 'name': 'Entities [En...","[{'type': 'replied_to', 'id': '160890909140099...",97133219,NaN
4,en,[1608533699745779715],3282033720,"{'annotations': [{'start': 183, 'end': 185, 'p...",1608533275256963072,everyone,@UWUnited_ @FIELHouston We continued our #NoTe...,1608533699745779715,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",2022-12-29T18:41:33.000Z,False,"[{'domain': {'id': '46', 'name': 'Business Tax...","[{'type': 'replied_to', 'id': '160853353645722...",3282033720,NaN
5,en,[1604761240517365760],3253750196,"{'annotations': [{'start': 47, 'end': 56, 'pro...",1604761240517365760,everyone,RT @BDSmovement: Israeli phone hacking company...,1604761240517365760,"{'retweet_count': 144, 'reply_count': 0, 'like...",2022-12-19T08:51:09.000Z,False,"[{'domain': {'id': '10', 'name': 'Person', 'de...","[{'type': 'retweeted', 'id': '1173629671977750...",NaN,NaN
6,en,[1603776218901647360],201864200,"{'annotations': [{'start': 19, 'end': 22, 'pro...",1603776218901647360,everyone,RT @athenaforall: $AMZN shareholders demand tr...,1603776218901647360,"{'retweet_count': 1, 'reply_count': 0, 'like_c...",2022-12-16T15:37:01.000Z,False,"[{'domain': {'id': '46', 'name': 'Business Tax...","[{'type': 'retweeted', 'id': '1603469315147239...",NaN,NaN
7,en,[1603602764537397250],59895033,"{'annotations': [{'start': 38, 'end': 44, 'pro...",1603602764537397250,everyone,pretty much every attempt to monetize twitter ...,1603602764537397250,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2022-12-16T04:07:47.000Z,False,"[{'domain': {'id': '30', 'name': 'Entities [En...",NaN,NaN,NaN
8,en,[1603479920012693504],1286446231,"{'annotations': [{'start': 19, 'end': 22, 'pro...",1603479920012693504,everyone,RT @athenaforall: $AMZN shareholders flag conc...,1603479920012693504,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",2022-12-15T19:59:38.000Z,False,"[{'domain': {'id': '46', 'name': 'Business Tax...","[{'type': 'retweeted', 'id': '1603472541389127...",NaN,NaN
9,en,[1603473598261432340],1362588085247238150,"{'annotations': [{'start': 19, 'end': 22, 'pro...",1603473598261432340,everyone,RT @athenaforall: $AMZN shareholders flag conc...,1603473598261432340,"{'retweet_count': 3, 'reply_count': 0, 'like_c...",2022-12-15T19:34:31.000Z,False,"[{'domain': {'id': '46', 'name': 'Business Tax...","[{'type': 'retweeted', 'id': '1603472541389127...",NaN,NaN


In [86]:
pd.DataFrame(tweet_flat).to_csv('tweets.csv')